In [ ]:
breast-cancer-classification/
│
├── data/                     # Dataset folder (not included in repo, add separately)
│   └── IDC_regular/          # Train & Test images
│
├── models/
│   └── cancer_net.h5         # Saved trained model (after training)
│
├── src/
│   ├── train.py              # Model training script
│   ├── evaluate.py           # Model evaluation & metrics
│   └── visualize.py          # Confusion matrix & learning curves
│
├── requirements.txt          # Python dependencies
└── README.md                 # Project documentation


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Paths to data
train_dir = "data/train"
test_dir = "data/test"

# Image Preprocessing
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_dir, target_size=(50, 50), batch_size=32, class_mode='binary')
test_gen = datagen.flow_from_directory(test_dir, target_size=(50, 50), batch_size=32, class_mode='binary')

# CNN Architecture (CancerNet)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(train_gen, epochs=10, validation_data=test_gen)

# Save model
model.save("models/cancer_net.h5")

# Evaluation
pred = model.predict(test_gen)
y_pred = np.round(pred).astype(int)
y_true = test_gen.classes

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

# Accuracy Plot
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title('Training & Validation Accuracy')
plt.show()
